# Assembly to dict

In [1]:
def dictionaryListCreatorPython(file):
    import sbol2 as sb2
    import json

    # Disable typed URIs for cleaner URI strings
    sb2.Config.setOption('sbol_typed_uris', False)

    # Read the SBOL document
    doc = sb2.Document()
    doc.read(file)

    # Known SO roles
    PRODUCT_ROLE = 'http://identifiers.org/so/SO:0000804'     # composite product
    PLASMID_ROLE = 'https://identifiers.org/SO:0000755'       # plasmid/backbone

    # SBOL type for Protein
    PROTEIN_TYPE = sb2.BIOPAX_PROTEIN  # URI for proteins

    product_dicts = []

    globalEnzyme = None
    for cd in doc.componentDefinitions:
        # Detect restriction enzyme by Protein type
        #create a filler variable
        if PROTEIN_TYPE in cd.types:
            globalEnzyme = cd.identity

        if PRODUCT_ROLE in cd.roles:
            #create new dict for every product
            result = {
                'Product': cd.identity,
                'Backbone': None,
                'PartsList': [],
                'Restriction Enzyme': None
            }

            for comp in cd.components:
                sub_def_uri = comp.definition
                sub_cd = doc.componentDefinitions.get(sub_def_uri)

                if sub_cd is None:
                    continue

                roles = sub_cd.roles
                display_id = sub_cd.displayId or ""

                #print(f"{display_id} roles: {roles} types: {types}")

                # Detect backbone by SO role
                if PLASMID_ROLE in roles:
                    result['Backbone'] = sub_def_uri
                    result['PartsList'].append(sub_def_uri)

                # Default case: treat as regular part
                else:
                    result['PartsList'].append(sub_def_uri)

            product_dicts.append(result)

        #add enzymes to each dictionary
        for dict in product_dicts:
            dict['Restriction Enzyme'] = globalEnzyme

    return product_dicts

In [2]:
dictionaryListCreatorPython("../tests/validation_assembly.xml")

[{'Product': 'https://SBOL2Build.org/composite_1/1',
  'Backbone': None,
  'PartsList': ['https://sbolcanvas.org/B0015/1',
   'https://sbolcanvas.org/Cir_qxow/1',
   'https://SBOL2Build.org/Ligation_Scar_A/1',
   'https://SBOL2Build.org/Ligation_Scar_D/1',
   'https://SBOL2Build.org/Ligation_Scar_B/1',
   'https://SBOL2Build.org/Ligation_Scar_E/1',
   'https://sbolcanvas.org/J23101/1',
   'https://sbolcanvas.org/GFP/1',
   'https://sbolcanvas.org/B0034/1',
   'https://SBOL2Build.org/Ligation_Scar_C/1'],
  'Restriction Enzyme': 'https://SBOL2Build.org/T4_Ligase/1'}]